<a href="https://colab.research.google.com/github/NusRAT-LiA/Llama-3.2-Medico-BD/blob/main/Fine_tune_3B_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine Tune

In [ ]:
%pip install -U bitsandbytes peft trl --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00


## 0. Download and Extract the Dataset

In [ ]:
!wget -nc https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/zhtvkny53n-1.zip
!unzip zhtvkny53n-1.zip

--2024-10-18 18:05:35--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/zhtvkny53n-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 3.5.72.122, 3.5.69.50, 52.218.36.243, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|3.5.72.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 268028 (262K) [application/zip]
Saving to: ‘zhtvkny53n-1.zip’

zhtvkny53n-1.zip    100%[===================>] 261.75K   335KB/s    in 0.8s    

2024-10-18 18:05:36 (335 KB/s) - ‘zhtvkny53n-1.zip’ saved [268028/268028]

Archive:  zhtvkny53n-1.zip
  inflating: Medicinal Products in Bangladesh - A Dataset of Generic and Brand Names, Dosage Strengths, and Manufacturers/Medicinal Products in Bangladesh A Dataset of Generic and Brand Names, Dosages, and Manufacturers.csv  


## 1. Libraries

In [ ]:
import os

import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoProcessor,
    TextStreamer,
    pipeline,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import json
import gc


__ = None

## 2. Model and Data paths

In [ ]:
model_path = "unsloth/Llama-3.2-3B-Instruct"
model_path_fine = 'llama-3.2-3B-finetuned'
data_path = "/content/Medicinal Products in Bangladesh - A Dataset of Generic and Brand Names, Dosage Strengths, and Manufacturers/Medicinal Products in Bangladesh A Dataset of Generic and Brand Names, Dosages, and Manufacturers.csv"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

## 3. QLoRA Params
This section of the configuration defines the hyperparameters for using LoRA (Low-Rank Adaptation) in a model, which is commonly applied to efficiently fine-tune large models by introducing trainable low-rank matrices into attention layers. The key parameters include:

- `lora_r`: Specifies the LoRA attention dimension or rank, which defines the rank of the low-rank matrices added to the model. A higher value increases model flexibility but also the number of trainable parameters.
  
- `lora_alpha`: This scaling parameter controls the magnitude of the LoRA updates by scaling the output of the low-rank matrices. A lower value restricts their influence on the model’s output, while a higher value amplifies their contribution.

- `lora_dropout`: The dropout probability applied to the LoRA weights during training to prevent overfitting. A value of `0.1` means there is a 10% chance that each LoRA weight will be dropped during training.

These parameters optimize memory and computation efficiency while maintaining the effectiveness of fine-tuning large models.

In [ ]:
lora_r = 64 #lora attention dimension/ rank
lora_alpha = 16 #lora scaling parameter
lora_dropout = 0.1 #lora dropout probability

## 4. BitsAndBytes Configurations
This configuration specifies settings for 4-bit quantization, a technique used to reduce the precision of model weights and activations, thereby improving memory efficiency and computational speed during inference or training, particularly in large models. The key parameters include:

- `use_4bit`: This flag indicates whether to use 4-bit quantization. When set to `True`, it enables the use of 4-bit precision for model parameters.

- `bnb_4bit_compute_dtype`: Specifies the data type used for computation when using 4-bit quantization. Here, it's set to `float16`, meaning the computations will still occur in 16-bit floating-point precision even though the weights are quantized to 4-bit.

- `bnb_4bit_quant_type`: Defines the type of quantization used. In this case, it's set to "nf4" (Normal Float 4), a specific quantization format known for preserving higher precision during quantization by focusing on the most significant bits.

- `use_nested_quant`: This flag determines whether nested quantization, a more advanced quantization technique, is applied. With `False`, nested quantization is disabled, meaning standard 4-bit quantization is used. This feature performs a second quantization of the already quantized weights to save an additional 0.4 bits/parameter.

These settings are crucial for optimizing memory use in models while maintaining an acceptable level of precision during computations.

In [ ]:
# Enable 4-bit quantization for memory efficiency
use_4bit = True

# Data type used for computation when using 4-bit quantization (here, float16 for precision)
bnb_4bit_compute_dtype = "float16"

# Type of 4-bit quantization used (nf4 for Normal Float 4, which helps maintain precision)
bnb_4bit_quant_type = "nf4"

# Flag to enable or disable nested quantization (disabled here)
use_nested_quant = False

## 5. Training Args
This configuration sets the essential hyperparameters and options required for training a model in PyTorch, defining the environment in which the model will operate. The `output_dir` specifies where model checkpoints will be stored during training. The `num_train_epochs` controls how many times the model will pass over the training data.

The floating-point precision is managed by the `fp16` and `bf16` flags, enabling mixed-precision training for efficiency on specific GPUs. Batch sizes for both training and evaluation are controlled by `per_device_train_batch_size` and `per_device_eval_batch_size`, ensuring proper memory allocation across GPUs.

Key optimization parameters include `learning_rate`, which dictates the step size in the gradient descent process, and `weight_decay`, helping to prevent overfitting. The `gradient_accumulation_steps` parameter allows for accumulation of gradients over several batches before updating the model, aiding training on smaller GPUs.

The `max_grad_norm` limits the gradient’s magnitude to prevent exploding gradients, while `optim` defines the optimizer used—here, the "paged_adamw_32bit". The learning rate scheduler, specified by `lr_scheduler_type`, ensures a gradual reduction in the learning rate during training, with a `warmup_ratio` to slowly ramp up learning at the start.

Finally, `seed` ensures reproducibility, and `group_by_length` optimizes batch formation by grouping sequences of similar lengths. The training process is monitored and checkpointed at intervals set by `save_steps` and `logging_steps`, respectively.

In [ ]:
# Directory where model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs (how many times the model will pass over the training data)
num_train_epochs = 1

# Enable fp16/bf16 training (fp16 or bf16 for mixed precision; set bf16 to True when using A100 GPU in Google Colab)
fp16 = False
bf16 = False

# Batch size per GPU for training (number of samples processed at once during training)
per_device_train_batch_size = 1

# Batch size per GPU for evaluation (number of samples processed at once during evaluation)
per_device_eval_batch_size = 1

# Gradient accumulation steps (number of steps to accumulate gradients before updating the model weights)
gradient_accumulation_steps = 1

# Learning rate (controls the step size during gradient descent)
learning_rate = 2e-4

# Weight decay (prevents overfitting by penalizing large weights)
weight_decay = 0.001

# Maximum gradient norm (gradient clipping to prevent exploding gradients)
max_grad_norm = 0.3

# Optimizer to use (paged_adamw_32bit for memory efficiency)
optim = "paged_adamw_32bit"

# Learning rate scheduler type (cosine schedule for gradual decay of the learning rate)
lr_scheduler_type = "cosine"

# Seed for reproducibility (ensures consistent results across runs)
seed = 1

# Maximum number of training steps (set to -1 for no specific limit)
max_steps = -1

# Ratio of steps for linear warmup (warmup adjusts learning rate in early stages of training)
warmup_ratio = 0.03

# Group sequences into batches of the same length (optimizes batch processing)
group_by_length = True

# Save checkpoint every X update steps (saves the model at regular intervals)
save_steps = 0

# Log training information every X update steps (controls frequency of logging)
logging_steps = 50

## 6. SFT (Supervised Fine Tuning) Parameters
This configuration extends the model setup by defining parameters related to sequence length, packing optimization, and GPU device allocation. The `max_seq_length` parameter sets the upper limit on the length of tokenized input sequences; when set to `None`, the model handles sequences up to its internal capacity without truncation.

The `packing` flag controls whether sequence packing is enabled, a technique that could help fit multiple shorter sequences into a fixed-length input, but here it is disabled (`False`).

Finally, the `device_map` ensures that the entire model is loaded onto GPU 0, optimizing memory and processing performance for training or inference on a single GPU setup.

In [ ]:
# Maximum sequence length for tokenized input (None means the model uses its internal limits)
max_seq_length = None

# Packing flag (set to False to disable sequence packing)
packing = False

# Device map for GPU allocation (the model is loaded entirely onto GPU 0)
device_map = {"": 0}

## 7. Dataset Loading
This code defines a custom dataset class `MedicineDataset` that extends PyTorch's `Dataset` class to handle medical data. The dataset is initialized with a pandas DataFrame, which contains relevant information such as the generic name, brand name, dosage type, strength, and manufacturer of medicines. The class provides two key methods:

- `__len__`: This method returns the total number of samples in the dataset, which corresponds to the number of rows in the DataFrame.
  
- `__getitem__`: This method retrieves a specific sample (based on an index) from the dataset and processes it into tokenized input using a tokenizer. The input text is formatted using information from various columns of the DataFrame. The tokenizer converts this input into token IDs with truncation and padding applied to ensure the input and labels have a fixed length of 128 tokens.

The output is a dictionary with two keys:
- `'input_ids'`: Tokenized input text representing a medicine's attributes.
- `'labels'`: Tokenized brand name used as the target output for supervised learning tasks.

In [ ]:
# Custom dataset class to handle medicine data for tokenization
class MedicineDataset(Dataset):
    def __init__(self, dataframe):
        # Initialize with a pandas DataFrame containing the medicine data
        self.data = dataframe

    def __len__(self):
        # Return the total number of samples (rows) in the dataset
        return len(self.data)

    def __getitem__(self, idx):
        # Fetch the row at the specified index
        row = self.data.iloc[idx]

        # Construct input text by concatenating various medicine details
        input_text = (f"Generic Name: {row['genericName']}, Brand Name: {row['brandName']}, "
                      f"Dosage Type: {row['dosageType']}, Strength: {row['strength']}, Manufacturer: {row['manufacturer']}")

        # Tokenize the input text and the brand name as labels, ensuring consistent length with truncation and padding
        return {
            'input_ids': tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=128)['input_ids'].squeeze(),
            'labels': tokenizer(row['brandName'], return_tensors='pt', truncation=True, padding='max_length', max_length=128)['input_ids'].squeeze()
        }

In [ ]:
# Load the dataset
df = pd.read_csv(data_path).dropna()
df.describe()

,genericName,brandName,packageMark,dosageType,strength,manufacturer
count,20604,20604,20604,20604,20604,20604
unique,1436,12957,20489,109,990,221
top,Cefixime Trihydrate,Renova,whitfieldointment63,Tablet,500 mg,Incepta Pharmaceuticals Ltd.
freq,341,9,4,9009,1252,1140


In [ ]:
# Create the dataset and the dataloader
dataset = MedicineDataset(df)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
# Split the dataset into train and eval
eval_split = 0.1
train_size = int(len(dataset) * (1 - eval_split))
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, eval_size])

## 8. Load Model

In [ ]:
# Load the tokenizer and the model with QLoRA config
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
compute_dtype

torch.float16

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = bnb_4bit_quant_type,
    bnb_4bit_compute_dtype = compute_dtype,
    bnb_4bit_use_double_quant = use_nested_quant
)
bnb_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [ ]:
# Checking GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("="*80)
        print("Your GPU supports bfloat16, you will get accelerated training with bf16 = True")
        print("="*80)

This section loads the base model for causal language modeling using the `AutoModelForCausalLM` class from Hugging Face's `transformers` library. The model is loaded from a pre-trained path and configured to use quantization and device mapping for efficient computation. Key components are:

- `model_path`: This specifies the path to the pre-trained model that will be loaded. It can point to a local directory or a model from the Hugging Face Model Hub.
  
- `quantization_config`: Passed as `bnb_config`, this argument configures the model for quantization, allowing efficient memory and computational use, especially for large models.
  
- `device_map`: Specifies how model layers are allocated across available devices (GPUs), which allows for distributing the model's components across multiple GPUs or a specific device like GPU 0.

After loading, two additional configurations are set:
- `use_cache`: Disabled by setting `model.config.use_cache = False`, which turns off the caching mechanism, often used during generation to store previous key/value states.
  
- `pretraining_tp`: Set to 1, this specifies the tensor parallelism factor during pretraining, controlling how the model divides its layers across devices. A value of 1 indicates no tensor parallelism.

These settings optimize the model for fine-tuning or inference in a resource-efficient way, particularly when using quantization.

In [ ]:
# Load the base model for causal language modeling with quantization and device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_path,  # Path to the pre-trained model
    quantization_config=bnb_config,  # Configuration for 4-bit quantization
    device_map=device_map,  # Map to allocate model layers to specific devices (GPUs)
)

# Disable caching mechanism to reduce memory usage during generation
model.config.use_cache = False

# Set pretraining tensor parallelism factor (1 means no tensor parallelism)
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

## 9. PEFT Config
This section defines the configuration for loading QLoRA (Quantized Low-Rank Adaptation) using the `LoraConfig` class, which is part of PEFT (Parameter-Efficient Fine-Tuning). QLoRA enables fine-tuning large language models in a memory-efficient way by applying low-rank updates to specific model parameters. The key configuration options include:

- `lora_alpha`: The scaling factor that controls the magnitude of LoRA updates. A lower value reduces their influence, while a higher value increases their effect on the model's output.
  
- `lora_dropout`: The dropout rate applied to LoRA layers during training, helping prevent overfitting. This value, set to `0.1`, drops 10% of the weights.

- `r`: Defines the rank of the low-rank matrices introduced by LoRA. Higher values add flexibility to the model but increase the number of trainable parameters.

- `bias`: Controls the handling of bias parameters in LoRA. In this case, it is set to `"none"`, indicating no modification of bias parameters during LoRA updates.

- `task_type`: Specifies the type of task the model is being fine-tuned for. Here, it is set to `"CAUSAL_LM"` (causal language modeling), making the setup suitable for autoregressive tasks where the model predicts the next token in a sequence.

These configurations set up QLoRA for fine-tuning causal language models in a resource-efficient manner while maintaining high performance.

In [ ]:
# Load QLoRA configuration using the LoraConfig class
peft_config = LoraConfig(
    lora_alpha=lora_alpha,       # Scaling factor for LoRA updates
    lora_dropout=lora_dropout,   # Dropout probability to prevent overfitting in LoRA layers
    r=lora_r,                    # LoRA attention rank (defines the low-rank update size)
    bias="none",                 # No modification of bias parameters in LoRA
    task_type="CAUSAL_LM",       # Task type set for causal language modeling
)

## 10. Training Loop

In [ ]:
#Set Training parameters
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim = optim,
    save_steps = save_steps,
    logging_steps = logging_steps,
    learning_rate = learning_rate,
    fp16 = fp16,
    bf16 = bf16,
    max_grad_norm = max_grad_norm,
    weight_decay = weight_decay,
    lr_scheduler_type = lr_scheduler_type,
    warmup_ratio = warmup_ratio,
    group_by_length = group_by_length,
    max_steps = max_steps,
    report_to = "tensorboard",
)

In [ ]:
#SFT Trainer
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    peft_config = peft_config,
    max_seq_length = max_seq_length,
    args = training_arguments,
    tokenizer = tokenizer,
    packing = packing,
)

#Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Step,Training Loss
50,3.182900


KeyboardInterrupt: 

## 11. Saving model

In [ ]:
# Save the trained model
trainer.model.save_pretrained(model_path_fine)

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [ ]:
prompt = "my tummy hurts, what medicine should i take?"

This code sets up a `TextStreamer` instace, which is used to stream the generated text from a model during the text generation process. The `TextStreamer` class allows real-time streaming of text output, especially useful for interactive applications like chatbots or live language generation.

Key parameters include:

- `tokenizer`: The tokenizer object is used to convert token IDs back into human-readable text. It handles decoding from tokens to words, ensuring proper format and spacing.

- `skip_prompt`: When set to `True`, this parameter ensures that the model's prompt (input text) is not included in the streamed output. Only the newly generated tokens will be displayed.

- `skip_special_tokens`: This flag ensures that special tokens (e.g., `[CLS]`, `[SEP]`, or other task-specific tokens) are removed from the output, resulting in cleaner text.

This setup enables the model to generate text in a live, interactive manner, with prompts and special tokens filtered out for a smooth user experience.


In [ ]:
# Initialize a TextStreamer for real-time text generation
streamer = TextStreamer(
    tokenizer,             # Tokenizer to decode token IDs back into text
    skip_prompt=True,      # Skip displaying the input prompt in the streamed output
    skip_special_tokens=True  # Exclude special tokens from the generated text
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

This code generates text using a pre-trained language model based on a given `prompt`, with several key parameters controlling the behavior of the text generation process. The generation process relies on the Hugging Face `generate` method and uses a `TextStreamer` to stream the output. Here's a breakdown of the key steps and parameters:

- `inputs`: The `tokenizer` converts the `prompt` into tokenized input tensors, with `return_tensors="pt"` indicating that the output will be in PyTorch tensor format. The `.to(device)` ensures that the inputs are moved to the correct device (CPU or GPU) for computation.

- `streamer`: The `TextStreamer` object streams the output text as it is generated, in real-time. It allows for interactive generation without waiting for the entire sequence to complete.

- `use_cache`: When set to `True`, caching of the key/value pairs in the attention layers is enabled, which speeds up the generation process by avoiding recalculating previous states.

- `max_new_tokens`: Limits the maximum number of new tokens to generate, ensuring that the output doesn't exceed the specified token count (1,000 tokens in this case).

- `do_sample`: Controls whether sampling is used during generation. When set to `False`, greedy decoding is used, meaning the model always selects the token with the highest probability. Setting it to `True` would enable sampling for more diverse outputs.

- `temperature`: Adjusts the randomness of token selection during generation. A value below 1.0 (0.7 in this case) makes the output more deterministic, reducing randomness in token selection.

- `repetition_penalty`: Penalizes repeated tokens during generation. A value of 1.1 slightly discourages the model from repeating the same tokens, leading to more diverse outputs.

This setup ensures that the model generates coherent, non-repetitive text with minimal randomness, while allowing for live streaming of the generated text.

In [ ]:
# Tokenize the prompt and move the input tensors to the appropriate device (CPU or GPU)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the output text using the model
outputs = model.generate(
    **inputs,               # Tokenized inputs for generation
    streamer=streamer,       # Stream the generated text in real-time
    use_cache=True,          # Enable caching for faster generation
    max_new_tokens=1000,     # Limit the generation to 1,000 new tokens
    do_sample=False,         # Disable sampling (use greedy decoding)
    temperature=None,        # Control randomness
    top_p=None,              # Control randomness
    repetition_penalty=1.1   # Penalize repeated tokens to encourage diversity
)

 for a child
For children, it's essential to choose medicines that are safe and suitable for their age group. Here are some general guidelines:

**For mild stomachaches:**

1.  **Antacids**: These can help neutralize stomach acid and provide quick relief. Examples include:
    *   Tums (calcium carbonate)
    *   Rolaids (aluminum hydroxide and magnesium hydroxide)
2.  **Ginger products**: Ginger has natural anti-inflammatory properties that may help soothe the stomach. You can try:
    *   Ginger ale or ginger tea
    *   Ginger candies or gummies
3.  **Over-the-counter pain relievers**: For older children (6+ years), you can consider:
    *   Acetaminophen (Tylenol) or ibuprofen (Advil or Motrin) under adult supervision

**Important notes:**

*   Always read and follow the label instructions carefully.
*   Consult with your pediatrician before giving any medication to a child.
*   Make sure to check the expiration dates and potency of any medication.
*   If your child experiences sev

## 12. Merge LoRA weight with Main Model

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, model_path_fine)
model = model.merge_and_unload()

# Reload the tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    streamer=streamer,
    use_cache=True,
    max_new_tokens=1000,
    do_sample=False,
    temperature=None,        # Control randomness
    top_p=None,              # Control randomness
    repetition_penalty=1.1
)

 for a child
For children, it's essential to choose medicines that are safe and suitable for their age group. Here are some general guidelines:

**For mild stomach pain:**

1.  **Acetaminophen (Tylenol)**: This is a common over-the-counter medication that can help relieve mild stomach pain in children.
2.  **Ibuprofen (Advil or Motrin)**: This medication can also be used to treat mild stomach pain, but only under the guidance of a pediatrician.

**Before giving any medication to your child:**

*   Always read the label carefully and follow the instructions.
*   Make sure you have the correct dosage and administration instructions.
*   Consult with your pediatrician if you're unsure about the best course of treatment.
*   Never give your child more than one medication at a time without consulting your pediatrician first.
*   Be aware of potential side effects and allergic reactions.

If your child experiences severe vomiting, bloody stools, or difficulty breathing, seek immediate medica

## 13. Unload the Current Model
This code snippet is responsible for freeing up memory by deleting model objects and clearing the GPU cache, particularly useful when dealing with large models that consume significant memory resources during training or inference. The key steps include:

- `del model`, `del trainer`, and `del base_model`: These lines explicitly delete the references to the model, trainer, and base model objects from memory. This is useful when you no longer need these objects and want to release the memory they occupy.

- `gc.collect()`: Invokes Python's garbage collector to clean up any unreferenced objects that are still lingering in memory. This step ensures that any memory that is still being held is released.

- `torch.cuda.empty_cache()`: If CUDA (GPU acceleration) is available, this method clears the memory allocated on the GPU by emptying the cache. While PyTorch automatically manages memory, this function can be called manually to free up unused GPU memory, especially when switching between large models or tasks.

This sequence of operations is particularly important when working with limited GPU resources or when running multiple models consecutively.

In [ ]:
# Explicitly delete model, trainer, and base model objects to free memory
del model
del trainer
del base_model

# Invoke Python's garbage collector to clean up unreferenced objects
gc.collect()

# If a CUDA-compatible GPU is available, clear the GPU memory cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## 14. Fine-tuned Model Inference
Due to time and resource constraints, the model above did not receive enough training. This section will demonstrate how inference in a model fine-tuned for a single epoch looks like.

In [ ]:
model_path_medico = 'cail-csedu/Llama-3.2-Medico'
prompt = prompt

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_path_medico,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

config.json:   0%|          | 0.00/937 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# Tokenize the prompt and move the input tensors to the appropriate device (CPU or GPU)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the output text using the model
outputs = model.generate(
    **inputs,               # Tokenized inputs for generation
    streamer=streamer,       # Stream the generated text in real-time
    use_cache=True,          # Enable caching for faster generation
    max_new_tokens=1000,     # Limit the generation to 1,000 new tokens
    do_sample=False,         # Disable sampling (use greedy decoding)
    temperature=None,        # Control randomness
    top_p=None,              # Control randomness
    repetition_penalty=1.1   # Penalize repeated tokens to encourage diversity
)

?
I'm not a doctor, so please keep in mind that this is not professional advice. If you're experiencing severe or persistent stomach pain, seek medical attention.

That being said, here are some common over-the-counter (OTC) medications for stomach issues:

1.  **Antacids**:
	* Tums
	* Rolaid
	* Mylanta
2.  **Anti-diarrheal**:
	* Imodium
	* Pepto-Bismol
3.  **Pain relievers**:
	* Acetaminophen (e.g., Tylenol)
	* Ibuprofen (e.g., Advil, Motrin)

When choosing a medication, consider the following factors:

*   The type of stomach issue: acid reflux, diarrhea, cramps, etc.
*   Your age and health status
*   Any underlying medical conditions
*   The dosage and instructions

Always read and follow the label, and consult with a healthcare professional if you're unsure about which medication to take or have any concerns.

If your symptoms persist or worsen, don't hesitate to reach out to a medical expert for personalized guidance.
